#Collect API Data from Deadlock-API Project, filter, format, and insert into Deadlock.db

#Requirements import from .txt
#Fetch data based on URL (site+endpoint)
##Fetch hero data
#Filter fetched data based on filters
##Filter hero data
#prints or formatting

#Requirements import from .txt

In [ ]:
!pip install -r requirements.txt

In [ ]:
import requests
import pandas as pd
import numpy as np

#Fetch data based on URL (site+endpoint), outputs DF

In [ ]:
#Fetches data and returns DF
def fetch_hero_data(url):
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        heroes_data = response.json()  # Converts the JSON response to a Python dictionary
    else:
        print(f"Failed to retrieve data: {response.status_code}")
    
    df = pd.DataFrame(heroes_data)
    return df

In [ ]:
##Fetch Hero data

In [ ]:

site = "https://assets.deadlock-api.com"
endpoint = "/v2/heroes/"
url = site+endpoint

heroes_df = fetch_hero_data(url)



##Filter Data

In [ ]:
def filter_hero_data(filters, df):
    filtered_data = []
    for _, item in df.iterrows():
        filtered_item = {key: item[key] for key in filters if key in item}
        filtered_data.append(filtered_item)
    
    return pd.DataFrame(filtered_data)


##Format Data

In [ ]:
def format_hero_data(df):
    df.rename(columns={'id' : 'hero_id'}, inplace=True)
    return df

##Filter and format Hero Data

In [ ]:

#columns to fetch
filters = ["id", "classname", "name", "description", "player_selectable", "disabled", "starting_stats", "level_info", "scaling_stats", "standard_level_up_upgrades"]

heroes_df = filter_hero_data(filters, heroes_df)
heroes_df = format_hero_data(heroes_df)

###Dispaly column headers.

In [19]:
# Display all columns
pd.set_option('display.max_columns', None)
heroes_df.head()  # Now it'll show all columns

,hero_id,name,description,player_selectable,disabled,starting_stats,level_info,scaling_stats,standard_level_up_upgrades
0,1,Infernus,{'lore': 'Like most teenagers; Infernus was wi...,True,False,"{'max_move_speed': {'value': 6.8, 'display_sta...",{'1': {'bonus_currencies': ['EAbilityUnlocks']...,{},{'MODIFIER_VALUE_BASE_BULLET_DAMAGE_FROM_LEVEL...
1,2,Seven,"{'lore': 'When mystic energy awoke on Earth, t...",True,False,"{'max_move_speed': {'value': 7.1, 'display_sta...",{'1': {'bonus_currencies': ['EAbilityUnlocks']...,{'EMaxMoveSpeed': {'scaling_stat': 'ETechPower...,{'MODIFIER_VALUE_BASE_BULLET_DAMAGE_FROM_LEVEL...
2,3,Vindicta,{'lore': 'A victim of John Hathorne during the...,True,False,"{'max_move_speed': {'value': 8.0, 'display_sta...",{'1': {'bonus_currencies': ['EAbilityUnlocks']...,"{'EFireRate': {'scaling_stat': 'ETechPower', '...",{'MODIFIER_VALUE_BASE_BULLET_DAMAGE_FROM_LEVEL...
3,4,Lady Geist,"{'lore': 'Once, Lady Jeanne Geist was the toas...",True,False,"{'max_move_speed': {'value': 6.3, 'display_sta...",{'1': {'bonus_currencies': ['EAbilityUnlocks']...,{},{'MODIFIER_VALUE_BASE_BULLET_DAMAGE_FROM_LEVEL...
4,6,Abrams,"{'lore': 'Hard hitting, hard headed, and hard ...",True,False,"{'max_move_speed': {'value': 6.8, 'display_sta...",{'1': {'bonus_currencies': ['EAbilityUnlocks']...,{},{'MODIFIER_VALUE_BASE_BULLET_DAMAGE_FROM_LEVEL...


###print header

In [ ]:
#print(heroes_df.head(1))

### Convert fitlered hero data to json

In [17]:
heroes_json = heroes_df.to_dict(orient='records')

### insert Hero data to database function

In [ ]:

def insert_hero_data_to_db(df,db):
    import sqlite3

    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    for hero in df:
        cursor.execute("""
            INSERT INTO heroes (hero_id, name, player_selectable, disabled)
            VALUES (?, ?, ?, ?)
        """, (
                 hero['hero_id'], 
            hero['name'],
            hero['player_selectable'], 
            hero['disabled'], 
        ))
    conn.commit()
    print("Data was successfully loaded")


### insert starting stats into stats table

In [ ]:
def insert_starting_stats():
    import sqlite3
    
    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    
    
    for hero in heroes_json:
        hero_id = hero["hero_id"]
        stats = hero.get("starting_stats", {})
    
        for stat_name, stat_obj in stats.items():
            base = stat_obj.get("value", None)
            if base is not None:
                cursor.execute("""
                    INSERT INTO hero_stats (hero_id, stat_name, base_value)
                    VALUES (?, ?, ?)
                """, (hero_id, stat_name, base))
                print(f"Expected row insertion for: {base}")
    conn.commit()

### Insert scaling stats into stats table, matching hero / stat

In [ ]:
def insert_scaling_stats():
    import sqlite3
    
    db = "Deadlock.db"
    
    #create or connect to database file
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    
    
    for hero in heroes_json:
        hero_id = hero["hero_id"]
        scaling_stats = hero.get("scaling_stats", {})
    
        for stat_name, scale_value in scaling_stats.items():
            cursor.execute("""
                UPDATE hero_stats
                SET scale_value = ?
                WHERE hero_id = ? AND stat_name = ?
            """, (scale_value, hero_id, stat_name))
    
    conn.commit()